# Project
### Preprocessing



In [11]:
import skimage.io as io
import numpy as np
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from commonfunctions import *
from pip._internal import main as install

In [18]:
img= cv2.imread("note.png") 
cv2.imshow("Images",img) 
cv2.waitKey(0) 
cv2.destroyAllWindows() 
countourImg=np.copy(img)
gray_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) 
#We May Calc NSR first before smoothing to avoid edge blurring in good quality images
smoothedImage=cv2.bilateralFilter(gray_img,8,10,10)
cv2.imshow("Images",smoothedImage) 
cv2.waitKey(0) 
cv2.destroyAllWindows() 


In [19]:
def customErosion(img,ST):
    countOnes= sum(sum(ST))
    imgConvolved = convolve2d(img,ST,mode="same")
    imgEroded = np.where(imgConvolved == countOnes,1,0)
    return imgEroded

def customDilation(img,ST):
    imgConvolved = convolve2d(img,ST,mode="same")
    imgDilated = np.where(imgConvolved >= 1,1,0)
    return imgDilated


In [20]:
def calcUb(img, imgHist, t):
    imgCopy = np.copy(img)
    size = img.shape[0]*img.shape[1]
    indeciesGreaterThanT = np.where(imgHist[1] >= t)
    indeciesLessThanT = np.where(imgHist[1] < t)
    indeciesGreaterThanT = np.array(indeciesGreaterThanT)
    indeciesLessThanT = np.array(indeciesLessThanT)
    grayLevelsGreater = np.zeros(indeciesGreaterThanT.shape[1])
    grayLevelsLess = np.zeros(indeciesLessThanT.shape[1])
    grayLevelsGreaterProb = np.zeros(indeciesGreaterThanT.shape[1])
    grayLevelsLessProb = np.zeros(indeciesLessThanT.shape[1])
    i = 0
    for index in indeciesGreaterThanT[0]:
        grayLevelsGreater[i] = imgHist[1][index]
        grayLevelsGreaterProb[i] = imgHist[0][index]
        i += 1
    i = 0
    for index in indeciesLessThanT[0]:
        grayLevelsLess[i] = imgHist[1][index]
        grayLevelsLessProb[i] = imgHist[0][index]
        i += 1
    size = img.shape[0]*img.shape[1]

    grayLevelsGreaterProb = grayLevelsGreaterProb/size
    grayLevelsLessProb = grayLevelsLessProb/size
    w1 = np.sum(grayLevelsLessProb)
    w2 = np.sum(grayLevelsGreaterProb)
    u1 = 0
    u2 = 0
    for x in range(grayLevelsLess.shape[0]):
        u1 += ((grayLevelsLess[x]*grayLevelsLessProb[x])/w1)
    for x in range(grayLevelsGreater.shape[0]):
        u2 += ((grayLevelsGreater[x]*grayLevelsGreaterProb[x])/w2)
    ub = w1*w2*(u1-u2)**2
    return ub


def OTSU_Binarization(grayImg):
    Threshold = 0
    Ub = 0
    imgHist = showHist(grayImg)
    for t in range(255):
        Ubtmp = calcUb(grayImg, imgHist, t)
        if(Ubtmp > Ub):
            Threshold = t
            Ub = Ubtmp
    img = np.where(grayImg >= Threshold, 0, 1)
    return img

In [21]:
BinarizedImage = OTSU_Binarization(smoothedImage)
cv2.imwrite('BinarizedImage.jpeg',BinarizedImage*255)
ST=np.ones((3,3))
eroded=customErosion(BinarizedImage,ST)
opened=customDilation(eroded,ST)
cv2.imwrite('Opened.jpg',opened*255)

True

In [22]:
countours,heirarchy=cv2.findContours(cv2.convertScaleAbs(BinarizedImage),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
cv2.drawContours(countourImg,countours,-1,(0,255,0))
cv2.imwrite('ContouredImg.jpg',countourImg)

True